#Setting up the environment

In [47]:
! pip install -q --upgrade google-generativeai langchain-google-genai chromadb pypdf python-dotenv

In [59]:
!pip install PyPDF2 langchain
!pip install -U langchain-community


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.7 MB/s eta 0:00:00


In [31]:
from IPython.display import display
from IPython.display import Markdown
import textwrap


def to_markdown(text):
  # text = response.candidates.content.get('parts', [{}])[0].get('text', 'Sorry, I am unable to answer your query at the moment.')
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [9]:
import google.generativeai as genai
from google.colab import userdata

In [12]:
import os
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)


#Text Generation

In [13]:
model = genai.GenerativeModel(model_name="gemini-pro")
model

genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
)

In [32]:
prompt = [
    "What is Mixture of Experts?",
]

response = model.generate_content(prompt)

In [39]:
to_markdown(response.text)

> There is no definitive answer to the question of the meaning of life, as it is a deeply personal and subjective matter. However, there are many different perspectives on this question, some of which include:
> 
> **1. The Purposeful Life:** This perspective suggests that life has an inherent purpose or goal, which can be discovered and pursued. This purpose may be religious, spiritual, or related to personal growth, achievement, or contribution to society.
> 
> **2. The Existentialist Life:** This perspective emphasizes the freedom and responsibility of individuals to create their own meaning in life. It suggests that life has no inherent purpose, and that individuals must take active steps to find meaning and fulfillment.
> 
> **3. The Hedonistic Life:** This perspective focuses on the pursuit of pleasure and the avoidance of pain as the primary goal of life. It suggests that meaning can be found in experiences that bring enjoyment and satisfaction.
> 
> **4. The Altruistic Life:** This perspective emphasizes the importance of helping others and making a positive contribution to the world. It suggests that meaning can be found in acts of compassion, kindness, and service to others.
> 
> **5. The Spiritual Life:** This perspective sees the meaning of life as being connected to a higher power, spiritual realm, or universal consciousness. It suggests that meaning can be found in practices of meditation, prayer, or other forms of spiritual connection.
> 
> Ultimately, the meaning of life is a question that each individual must answer for themselves. There is no right or wrong answer, and what is meaningful to one person may not be meaningful to another. However, by exploring different perspectives and engaging in self-reflection, individuals can come to a better understanding of their own values, beliefs, and aspirations, and ultimately discover what gives their life meaning and purpose.

#In Context Information Retrieval

In [60]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.document_loaders import PyPDFLoader


In [ ]:
model = ChatGoogleGenerativeAI(model_name="gemini-pro",temperature=0.3)

#Exctract text from pdf

In [72]:
pdf_loader = PyPDFLoader("https://images.collegedunia.com/public/image/cd430f5d18c356cacfbec5c0b48c91e7.pdf")
pages = pdf_loader.load_and_split()
print(pages[37].page_content)

ValueError: Check the url of your file; returned status code 401

In [73]:
print(pages[37].page_content)

22 Chemistry
∴ Packing   efficiency
= Volumeof one atom100%Volumeof cubic unit cell×
= 3
34r3100 1006 8rππ× = ×
= 52.36% = 52.4 %
Thus, we may conclude that ccp
and hcp structures have maximum
packing efficiency.
From the unit cell dimensions, it is possible to calculate the volume of
the unit cell. Knowing the density of the metal, we can calculate the
mass of the atoms in the unit cell. The determination of the mass of a
single atom gives an accurate method of determination of Avogadro
constant . Suppose, edge length of a unit cell of a cubic crystal
determined by X-ray diffraction is a, d the density of the solid substance
and M the molar mass. In  case of cubic crystal:
Volume of a unit cell = a3
Mass of the unit cell
= number of atoms in unit cell × mass of each atom = z × m
(Here z is the number of atoms present in one unit cell and m is the
mass of a single atom)
Mass of an atom present in the unit cell:
m 
AM=N(M is molar mass)
Therefore, density of the unit cell
  3 3 3
A Amas

In [66]:
len(pages)

305

In [70]:
context = "\n".join(str(p.page_content) for p in pages[111:133])
print("The total words in the context: ", len(context))

The total words in the context:  40617


#Prompt Design- In Context

In [77]:
from langchain.prompts import PromptTemplate
from langchain.llms import load_qa_chain

ImportError: cannot import name 'load_qa_chain' from 'langchain.llms' (/usr/local/lib/python3.10/dist-packages/langchain/llms/__init__.py)

In [75]:
prompt_template = """Answer the question as precise as possible using the provided context. If the answer is
                    not contained in the context, say "Answer not available in context." \n\n
                    Context: \n {context}?\n
                    Question: \n {question} \n
                    Answer:
                  """

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [76]:
stuff_chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

NameError: name 'load_qa_chain' is not defined